**Install the RLDS and other required modules**

In [ ]:
!pip install rlds[tensorflow]
!pip install tfds-nightly

In [ ]:
!pip install apache-beam

In [86]:
from typing import Any, Dict, Union, NamedTuple

import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import rlds

# **Choose an RLDS dataset - Choosing D4RL MuJoCo Hopper as an example**

In [87]:
dataset_name = 'd4rl_mujoco_hopper'  # @param { isTemplate: true}
num_episodes_to_load = 10   # @param { isTemplate: true}

In [88]:
dataset, info = tfds.load(dataset_name, with_info=True)

**Print information about the dataset and its format**

In [89]:
print(info)

tfds.core.DatasetInfo(
    name='d4rl_mujoco_hopper',
    full_name='d4rl_mujoco_hopper/v0-expert/1.2.0',
    description="""
    D4RL is an open-source benchmark for offline reinforcement learning. It provides
    standardized environments and datasets for training and benchmarking algorithms.
    
    The datasets follow the [RLDS format](https://github.com/google-research/rlds)
    to represent steps and episodes.
    """,
    config_description="""
    See more details about the task and its versions in https://github.com/rail-berkeley/d4rl/wiki/Tasks#gym
    """,
    homepage='https://sites.google.com/view/d4rl-anonymous',
    data_dir='/root/tensorflow_datasets/d4rl_mujoco_hopper/v0-expert/1.2.0',
    file_format=tfrecord,
    download_size=51.56 MiB,
    dataset_size=64.10 MiB,
    features=FeaturesDict({
        'steps': Dataset({
            'action': Tensor(shape=(3,), dtype=float32),
            'discount': float32,
            'is_first': bool,
            'is_last': bool,


**View Episode and Step structure - specifically the actions**

In [90]:
# Single episode from the dataset

## Format of the dataset is RLDS:
## Episode: [step: [action, reward, observation,...], step, step...], Episode, Episode...
## To understand how actions, observations, and reward are represented, refer to MuJoCo Hopper docs: https://gymnasium.farama.org/environments/mujoco/hopper/


print(dataset['train'])

episode = next(iter(dataset['train']))

# Print episode structure
print("\nEpisode Structure:")
print(episode.items())


for item in episode.items():

    #Using numpy_iterator to query the elements in the _VariantDataset element_spec data format
    ele  = item[1].as_numpy_iterator()
    ele_iterated = np.array([val for val in ele])
    #print(ele_iterated)
    print("\n")
    print(len(ele_iterated))
    print('\naction:')
    print(ele_iterated[0]['action'])
    print(len(ele_iterated[0]['action']))
    print('\nobservation:')
    print(ele_iterated[0]['observation'])
    print(len(ele_iterated[0]['observation']))
    print('\nreward:')
    print(ele_iterated[0]['reward'])


<_PrefetchDataset element_spec={'steps': DatasetSpec({'action': TensorSpec(shape=(3,), dtype=tf.float32, name=None), 'discount': TensorSpec(shape=(), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'observation': TensorSpec(shape=(11,), dtype=tf.float32, name=None), 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorShape([]))}>

Episode Structure:
dict_items([('steps', <_VariantDataset element_spec={'action': TensorSpec(shape=(3,), dtype=tf.float32, name=None), 'discount': TensorSpec(shape=(), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'observation': TensorSpec(shape=(11,), dtype=tf.float32, name=None), 'reward': TensorSpec(shape=(), dtype=t

# **Choose an HF JAT dataset - Choosing MuJoCo Hopper as an example**

In [ ]:
!pip install datasets

In [92]:
from datasets import load_dataset

In [93]:
dataset_hf = load_dataset("jat-project/jat-dataset", "mujoco-hopper")

**Check structure of HF dataset**

In [94]:
print(dataset_hf['train'])

Dataset({
    features: ['continuous_observations', 'continuous_actions', 'rewards'],
    num_rows: 9000
})


In [95]:
first_episode = dataset_hf["train"][0]
print(first_episode)

{'continuous_observations': [[1.2464417219161987, -0.0026244299951940775, -0.0025811174418777227, -0.0036725590471178293, -0.003424909897148609, 0.003843150567263365, 0.003587322076782584, 0.0005191994714550674, 0.0013864905340597034, -0.0046056038700044155, -0.00010990180453518406], [1.2460367679595947, 0.0005410704761743546, 0.00165370583999902, -0.005018089897930622, 0.0023089128080755472, 0.0480564683675766, -0.10440516471862793, 0.5534252524375916, 0.7638858556747437, -0.30699360370635986, 1.4351338148117065], [1.2447866201400757, 0.002800705609843135, 0.005820226855576038, -0.008984911255538464, 0.019592761993408203, 0.01956210844218731, -0.20819228887557983, 0.047571953386068344, 0.32253462076187134, -0.6877645254135132, 2.8834383487701416], [1.242730975151062, 0.0001934929605340585, 0.007343162316828966, -0.019367476925253868, 0.04859178885817528, -0.07497024536132812, -0.3066785931587219, -0.6775923371315002, 0.08471795171499252, -1.9085766077041626, 4.363751411437988], [1.239

**Converting the HF JAT dataset to the TFDS format**

In [ ]:
!pip install tensorflow_datasets

In [97]:
from tensorflow_datasets import dataset_builders

In [98]:
tf_ds = dataset_builders.HuggingfaceDatasetBuilder(hf_repo_id= "jat-project/jat-dataset", hf_config= "mujoco-hopper")

In [99]:
tf_ds.download_and_prepare()

In [100]:
tf_ds_converted = tf_ds.as_dataset()

**View Episode and Step structure - specifically the actions**

In [103]:
## Format of the dataset:
## Episode: {continuous_actions: action x num_steps, continuous_observations: observation x num_steps, rewards: reward x num_steps}, Episode, Episode,...
## To understand how actions, observations, and reward are represented, refer to MuJoCo Hopper docs: https://gymnasium.farama.org/environments/mujoco/hopper/

episode_tfds = next(iter(tf_ds_converted['train']))
print(len(episode_tfds))

# Print episode structure
print("\nEpisode Structure:\n")

for item in episode_tfds.items():

    print(item[0])
    print(item[1][0])
    print(item[1][0].shape)


3

Episode Structure:

continuous_actions
tf.Tensor([ 0.7731227  -0.12329477  1.4207025 ], shape=(3,), dtype=float32)
(3,)
continuous_observations
tf.Tensor(
[ 1.24644172e+00 -2.62443000e-03 -2.58111744e-03 -3.67255905e-03
 -3.42490990e-03  3.84315057e-03  3.58732208e-03  5.19199471e-04
  1.38649053e-03 -4.60560387e-03 -1.09901805e-04], shape=(11,), dtype=float32)
(11,)
rewards
tf.Tensor(1.0301455, shape=(), dtype=float32)
()
